# 라이브러리 및 데이터

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install tensorflow

In [3]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [4]:
x_data = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/2. 상권 활성화 지수 예측 모델/0) 데이터/data_x.csv')
y_data = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/2. 상권 활성화 지수 예측 모델/0) 데이터/data_y.csv')

In [5]:
scaler = MinMaxScaler()
x_columns_to_scale = x_data.columns.difference(['년도', '분기', '행정동코드', '표준산업분류코드'])
y_columns_to_scale = y_data.columns.difference(['년도', '분기', '행정동코드', '표준산업분류코드'])

x_data[x_columns_to_scale] = scaler.fit_transform(x_data[x_columns_to_scale])
y_data[y_columns_to_scale] = scaler.fit_transform(y_data[y_columns_to_scale])

# 모델링

## Linear Regression

In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values = pd.DataFrame(index=x_data.index)

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = LinearRegression()

    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE: {mse}')

    predicted_values[f'{predict_column}_1_pre'] = model.predict(x_data)

lr_pred = pd.concat([x_data, predicted_values], axis=1)

grade 예측 MSE: 0.0002493775547587891
result 예측 MSE: 0.001868943566274145
sales 예측 MSE: 0.0005951505915599874
infra 예측 MSE: 0.0003160802922126036
franch 예측 MSE: 0.003541150432148625
pop 예측 MSE: 0.00045607666531934424
fin 예측 MSE: 0.00018206938226202868


## XGBoost

In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values = pd.DataFrame(index=x_data.index)

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = XGBRegressor(random_state=42)

    param_grid = {'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]}

    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(x_train, y_train)

    best_n_estimators = grid_search.best_params_['n_estimators']
    print(f'Best n_estimators for {predict_column}: {best_n_estimators}')

    final_model = XGBRegressor(n_estimators=best_n_estimators, random_state=42)
    final_model.fit(x_train, y_train)

    y_pred = final_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE: {mse}')

    predicted_values[f'{predict_column}_1_pre'] = final_model.predict(x_data)

xgb_pred = pd.concat([x_data, predicted_values], axis=1)

Best n_estimators for grade: 600
grade 예측 MSE: 0.00013817806531693437
Best n_estimators for result: 200
result 예측 MSE: 0.0020548436539918977
Best n_estimators for sales: 200
sales 예측 MSE: 0.00043229000024283846
Best n_estimators for infra: 200
infra 예측 MSE: 0.00024023010634554903
Best n_estimators for franch: 100
franch 예측 MSE: 0.00095284021618859
Best n_estimators for pop: 200
pop 예측 MSE: 0.0006650815354796777
Best n_estimators for fin: 400
fin 예측 MSE: 9.079994387582427e-05


## RandomForest

In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra']

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = RandomForestRegressor(random_state=42)

    param_grid = {'n_estimators': [100, 200, 300, 400, 500]}

    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(x_train, y_train)

    best_n_estimators = grid_search.best_params_['n_estimators']
    print(f'Best n_estimators for {predict_column}: {best_n_estimators}')

    final_model = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)
    final_model.fit(x_train, y_train)

    y_pred = final_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE: {mse}')

Best n_estimators for grade: 500
grade 예측 MSE: 0.00016896106524057298
Best n_estimators for result: 400
result 예측 MSE: 0.0024163131826768548
Best n_estimators for sales: 300
sales 예측 MSE: 0.00035926752020933977
Best n_estimators for infra: 400
infra 예측 MSE: 0.00016086873406102927


In [ ]:
target_columns = ['franch', 'pop', 'fin']

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = RandomForestRegressor(random_state=42)

    param_grid = {'n_estimators': [100, 200, 300, 400, 500]}

    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(x_train, y_train)

    best_n_estimators = grid_search.best_params_['n_estimators']
    print(f'Best n_estimators for {predict_column}: {best_n_estimators}')

    final_model = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)
    final_model.fit(x_train, y_train)

    y_pred = final_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE: {mse}')

Best n_estimators for franch: 100
franch 예측 MSE: 0.0010449255301962792
Best n_estimators for pop: 500
pop 예측 MSE: 0.0009865179465192544
Best n_estimators for fin: 400
fin 예측 MSE: 0.0001111237123514923


In [ ]:
target_columns = ['grade', 'pop']

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = RandomForestRegressor(random_state=42)

    param_grid = {'n_estimators': [600, 700, 800, 900, 1000]}

    grid_search = GridSearchCV(model, param_grid, scoring='neg_mean_squared_error', cv=3)
    grid_search.fit(x_train, y_train)

    best_n_estimators = grid_search.best_params_['n_estimators']
    print(f'Best n_estimators for {predict_column}: {best_n_estimators}')

    final_model = RandomForestRegressor(n_estimators=best_n_estimators, random_state=42)
    final_model.fit(x_train, y_train)

    y_pred = final_model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE: {mse}')

Best n_estimators for grade: 1000
grade 예측 MSE: 0.0001711757363007149
Best n_estimators for pop: 600
pop 예측 MSE: 0.0009907963275322116


In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']
n_estimators_list = [1000, 400, 300, 400, 100, 600, 400]

predicted_values = pd.DataFrame(index=x_data.index)

for i, predict_column in enumerate(target_columns):
    n_estimators = n_estimators_list[i]

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
    model.fit(x_train, y_train)

    y_pred = model.predict(x_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f'{predict_column} 예측 MSE (n_estimators={n_estimators}): {mse}')

    predicted_values[f'{predict_column}_1_pre'] = model.predict(x_data)

rf_pred = pd.concat([x_data, predicted_values], axis=1)

grade 예측 MSE (n_estimators=1000): 0.0001711757363007149
result 예측 MSE (n_estimators=400): 0.0024163131826768548
sales 예측 MSE (n_estimators=300): 0.00035926752020933977
infra 예측 MSE (n_estimators=400): 0.00016086873406102927
franch 예측 MSE (n_estimators=100): 0.0010449255301962792
pop 예측 MSE (n_estimators=600): 0.0009907963275322116
fin 예측 MSE (n_estimators=400): 0.0001111237123514923


## NeuralNetwork

In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_1 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        Dense(50, input_shape=(x_train.shape[1],)),
        BatchNormalization(),
        Activation('relu'),
        Dense(50),
        BatchNormalization(),
        Activation('relu'),
        Dense(1),
        Activation('sigmoid'),
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint_filepath = f"best_model_checkpoint1_{predict_column}_1.h5"

    model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True)

    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint], verbose=1)

    model.load_weights(checkpoint_filepath)

    y_pred = model.predict(x_test).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    predicted_values_1[f'{predict_column}_1_pre'] = model.predict(x_data).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

nn_pred_1 = pd.concat([x_data, predicted_values_1], axis=1)

Epoch 1/100
776/776 [==============================] - 8s 5ms/step - loss: 0.0201 - val_loss: 0.1554
Epoch 2/100
 40/776 [>.............................] - ETA: 2s - loss: 0.0169

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 4s 6ms/step - loss: 0.0180 - val_loss: 0.0639
Epoch 3/100
776/776 [==============================] - 3s 4ms/step - loss: 0.0180 - val_loss: 0.0737
Epoch 4/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0179 - val_loss: 0.0437
Epoch 5/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0178 - val_loss: 0.0962
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0176 - val_loss: 0.0363
Epoch 7/100
776/776 [==============================] - 3s 4ms/step - loss: 0.0177 - val_loss: 0.1307
Epoch 8/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0176 - val_loss: 0.2445
Epoch 9/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0177 - val_loss: 0.0639
Epoch 10/100
776/776 [==============================] - 3s 4ms/step - loss: 0.0176 - val_loss: 0.0650
Epoch 11/100
776/776 [==============================] - 3s 4ms/step - loss: 0.0177 - val_loss: 0.0838


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 4s 5ms/step - loss: 0.0843 - val_loss: 0.1179
Epoch 3/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0835 - val_loss: 0.0873
Epoch 4/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0834 - val_loss: 0.2116
Epoch 5/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0834 - val_loss: 0.0969
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0835 - val_loss: 0.1058
Epoch 7/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0831 - val_loss: 0.2470
Epoch 8/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0829 - val_loss: 0.1909
Epoch 9/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0829 - val_loss: 0.2415
Epoch 10/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0830 - val_loss: 0.1397
Epoch 11/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0831 - val_loss: 0.4288


...

776/776 [==============================] - 4s 5ms/step - loss: 0.0225 - val_loss: 0.0239
Epoch 3/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0225 - val_loss: 0.0149
Epoch 4/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0225 - val_loss: 0.0192
Epoch 5/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0224 - val_loss: 0.0442
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0223 - val_loss: 0.1462
Epoch 7/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0223 - val_loss: 0.0807
Epoch 8/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0223 - val_loss: 0.2008
Epoch 9/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0223 - val_loss: 0.0333
Epoch 10/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0223 - val_loss: 0.0165
Epoch 11/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0223 - val_loss: 0.0736


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 4s 5ms/step - loss: 0.0241 - val_loss: 0.1447
Epoch 3/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0240 - val_loss: 0.0251
Epoch 4/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0240 - val_loss: 0.0440
Epoch 5/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0239 - val_loss: 0.1230
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0239 - val_loss: 0.0396
Epoch 7/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0239 - val_loss: 0.0339
Epoch 8/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0239 - val_loss: 0.0332
Epoch 9/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0238 - val_loss: 0.0341
Epoch 10/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0239 - val_loss: 0.0286
Epoch 11/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0238 - val_loss: 0.0270


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 3s 4ms/step - loss: 0.0156 - val_loss: 0.0403
Epoch 3/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0155 - val_loss: 0.0422
Epoch 4/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0154 - val_loss: 0.0552
Epoch 5/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0155 - val_loss: 0.0215
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0154 - val_loss: 0.0505
Epoch 7/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0154 - val_loss: 0.0283
Epoch 8/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0155 - val_loss: 0.0319
Epoch 9/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0154 - val_loss: 0.0264
Epoch 10/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0153 - val_loss: 0.0722
Epoch 11/100
243/243 [==============================] - 0s 2ms/step
fin 예측 MSE: 0.021006623848326817
1

In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_2 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        Dense(64, input_shape=(x_train.shape[1],)),
        BatchNormalization(),
        Activation('relu'),
        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dense(16),
        BatchNormalization(),
        Activation('relu'),
        Dense(1),
        Activation('sigmoid'),
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint_filepath = f"best_model_checkpoint2_{predict_column}_1.h5"

    model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True)
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint], verbose=1)

    model.load_weights(checkpoint_filepath)

    y_pred = model.predict(x_test).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    predicted_values_2[f'{predict_column}_1_pre'] = model.predict(x_data).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

nn_pred_2 = pd.concat([x_data, predicted_values_2], axis=1)

Epoch 1/100
776/776 [==============================] - 17s 9ms/step - loss: 0.0182 - val_loss: 0.0270
Epoch 2/100
 17/776 [..............................] - ETA: 5s - loss: 0.0144

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 6ms/step - loss: 0.0175 - val_loss: 0.0284
Epoch 3/100
776/776 [==============================] - 7s 8ms/step - loss: 0.0174 - val_loss: 0.0809
Epoch 4/100
776/776 [==============================] - 7s 9ms/step - loss: 0.0173 - val_loss: 0.0679
Epoch 5/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0173 - val_loss: 0.0376
Epoch 6/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0172 - val_loss: 0.0219
Epoch 7/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0169 - val_loss: 0.0343
Epoch 8/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0169 - val_loss: 0.0585
Epoch 9/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0169 - val_loss: 0.0497
Epoch 10/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0168 - val_loss: 0.0353
Epoch 11/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0168 - val_loss: 0.0264


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 6ms/step - loss: 0.0832 - val_loss: 0.2362
Epoch 3/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0828 - val_loss: 0.2987
Epoch 4/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0826 - val_loss: 0.2102
Epoch 5/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0820 - val_loss: 0.1679
Epoch 6/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0821 - val_loss: 0.1907
Epoch 7/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0817 - val_loss: 0.3006
Epoch 8/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0815 - val_loss: 0.1243
Epoch 9/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0814 - val_loss: 0.1881
Epoch 10/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0817 - val_loss: 0.3030
Epoch 11/100
243/243 [==============================] - 0s 2ms/step
result 예측 MSE: 0.12270411705842342

...

776/776 [==============================] - 5s 7ms/step - loss: 0.0225 - val_loss: 0.0402
Epoch 3/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0223 - val_loss: 0.0523
Epoch 4/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0222 - val_loss: 0.0361
Epoch 5/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0361
Epoch 6/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0222 - val_loss: 0.0241
Epoch 7/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0221 - val_loss: 0.0386
Epoch 8/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0150
Epoch 9/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0221 - val_loss: 0.0197
Epoch 10/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0221 - val_loss: 0.0679
Epoch 11/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0221 - val_loss: 0.0403


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 6ms/step - loss: 0.0240 - val_loss: 0.0571
Epoch 3/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0239 - val_loss: 0.0663
Epoch 4/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0238 - val_loss: 0.0703
Epoch 5/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0238 - val_loss: 0.0495
Epoch 6/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0238 - val_loss: 0.0886
Epoch 7/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0237 - val_loss: 0.0558
Epoch 8/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0238 - val_loss: 0.0239
Epoch 9/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0237 - val_loss: 0.0385
Epoch 10/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0237 - val_loss: 0.0669
Epoch 11/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0237 - val_loss: 0.0627


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 6ms/step - loss: 0.0152 - val_loss: 0.0208
Epoch 3/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0151 - val_loss: 0.0596
Epoch 4/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0151 - val_loss: 0.0399
Epoch 5/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0150 - val_loss: 0.0816
Epoch 6/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0149 - val_loss: 0.0276
Epoch 7/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0149 - val_loss: 0.0350
Epoch 8/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0148 - val_loss: 0.0307
Epoch 9/100
776/776 [==============================] - 4s 6ms/step - loss: 0.0147 - val_loss: 0.0399
Epoch 10/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0147 - val_loss: 0.0227
Epoch 11/100
776/776 [==============================] - 4s 5ms/step - loss: 0.0147 - val_loss: 0.0515


In [ ]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_3 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train, x_test = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        Dense(64, input_shape=(x_train.shape[1],)),
        BatchNormalization(),
        Activation('relu'),
        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dense(16),
        BatchNormalization(),
        Activation('relu'),
        Dense(8),
        BatchNormalization(),
        Activation('relu'),
        Dense(1),
        Activation('sigmoid'),
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    checkpoint_filepath = f"best_model_checkpoint3_{predict_column}_1.h5"

    model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True)
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint], verbose=1)

    model.load_weights(checkpoint_filepath)

    y_pred = model.predict(x_test).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    predicted_values_3[f'{predict_column}_1_pre'] = model.predict(x_data).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

nn_pred_3 = pd.concat([x_data, predicted_values_3], axis=1)

Epoch 1/100
776/776 [==============================] - 8s 7ms/step - loss: 0.0189 - val_loss: 0.0550
Epoch 2/100
 19/776 [..............................] - ETA: 4s - loss: 0.0166

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 6s 8ms/step - loss: 0.0174 - val_loss: 0.0340
Epoch 3/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0172 - val_loss: 0.0633
Epoch 4/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0171 - val_loss: 0.0649
Epoch 5/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0171 - val_loss: 0.0263
Epoch 6/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0169 - val_loss: 0.0409
Epoch 7/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0169 - val_loss: 0.0387
Epoch 8/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0168 - val_loss: 0.0494
Epoch 9/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0167 - val_loss: 0.0404
Epoch 10/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0166 - val_loss: 0.0602
Epoch 11/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0167 - val_loss: 0.0589


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 6ms/step - loss: 0.0828 - val_loss: 0.4306
Epoch 3/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0819 - val_loss: 0.1602
Epoch 4/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0823 - val_loss: 0.1465
Epoch 5/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0817 - val_loss: 0.2044
Epoch 6/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0816 - val_loss: 0.1120
Epoch 7/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0817 - val_loss: 0.1866
Epoch 8/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0816 - val_loss: 0.0999
Epoch 9/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0816 - val_loss: 0.1207
Epoch 10/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0813 - val_loss: 0.2336
Epoch 11/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0814 - val_loss: 0.1312


...

776/776 [==============================] - 5s 7ms/step - loss: 0.0226 - val_loss: 0.1276
Epoch 3/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0224 - val_loss: 0.0844
Epoch 4/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0508
Epoch 5/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0263
Epoch 6/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0221 - val_loss: 0.0381
Epoch 7/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0312
Epoch 8/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0222 - val_loss: 0.0372
Epoch 9/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0222 - val_loss: 0.0189
Epoch 10/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0221 - val_loss: 0.0387
Epoch 11/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0221 - val_loss: 0.0450


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 5s 7ms/step - loss: 0.0239 - val_loss: 0.1960
Epoch 3/100
776/776 [==============================] - 7s 9ms/step - loss: 0.0238 - val_loss: 0.0306
Epoch 4/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0237 - val_loss: 0.0551
Epoch 5/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0237 - val_loss: 0.0243
Epoch 6/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0237 - val_loss: 0.0292
Epoch 7/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0237 - val_loss: 0.0230
Epoch 8/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0237 - val_loss: 0.0251
Epoch 9/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0237 - val_loss: 0.0250
Epoch 10/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0236 - val_loss: 0.0283
Epoch 11/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0236 - val_loss: 0.0401


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


776/776 [==============================] - 6s 8ms/step - loss: 0.0153 - val_loss: 0.0228
Epoch 3/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0151 - val_loss: 0.1074
Epoch 4/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0150 - val_loss: 0.0217
Epoch 5/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0149 - val_loss: 0.0236
Epoch 6/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0147 - val_loss: 0.0212
Epoch 7/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0147 - val_loss: 0.0291
Epoch 8/100
776/776 [==============================] - 5s 6ms/step - loss: 0.0147 - val_loss: 0.1329
Epoch 9/100
776/776 [==============================] - 6s 8ms/step - loss: 0.0146 - val_loss: 0.0270
Epoch 10/100
776/776 [==============================] - 5s 7ms/step - loss: 0.0146 - val_loss: 0.0512
Epoch 11/100
776/776 [==============================] - 6s 7ms/step - loss: 0.0146 - val_loss: 0.0445


## LSTM

In [12]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_1 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    x_data_temp = x_data[[f'{predict_column}_2', f'{predict_column}_3']]

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train_current, x_test_current = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        LSTM(64, input_shape=(x_train_current.shape[1], 1), activation='relu'),
        BatchNormalization(),
        Dense(32),
        BatchNormalization(),
        Activation('relu'),
        Dense(1),
        Activation('linear'),
    ])

    print("형태 변경 전 - x_train_current 형태:", x_train_current.shape)
    x_train_lstm_current = np.reshape(x_train_current.values, (x_train_current.shape[0], x_train_current.shape[1], 1))
    x_test_lstm_current = np.reshape(x_test_current.values, (x_test_current.shape[0], x_test_current.shape[1], 1))
    print("형태 변경 후 - x_train_lstm_current 형태:", x_train_lstm_current.shape)

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(x_train_lstm_current, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

    y_pred = model.predict(x_test_lstm_current).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    x_data_lstm_current = np.reshape(x_data_temp.values, (x_data_temp.shape[0], x_data_temp.shape[1], 1))
    predicted_values_1[f'{predict_column}_1_pre'] = model.predict(x_data_lstm_current).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

lstm_pred_1 = pd.concat([x_data, predicted_values_1], axis=1)

형태 변경 전 - x_train_current 형태: (31026, 88)
형태 변경 후 - x_train_lstm_current 형태: (31026, 88, 1)
Epoch 1/50
776/776 [==============================] - 33s 39ms/step - loss: 0.1989 - val_loss: 0.0255
Epoch 2/50
776/776 [==============================] - 31s 40ms/step - loss: 0.0200 - val_loss: 0.0439
Epoch 3/50
776/776 [==============================] - 32s 42ms/step - loss: 0.0196 - val_loss: 0.0194
Epoch 4/50
776/776 [==============================] - 32s 42ms/step - loss: 0.0193 - val_loss: 0.0191
Epoch 5/50
776/776 [==============================] - 30s 39ms/step - loss: 0.0192 - val_loss: 0.0227
Epoch 6/50
776/776 [==============================] - 33s 42ms/step - loss: 0.0191 - val_loss: 0.0180
Epoch 7/50
776/776 [==============================] - 30s 39ms/step - loss: 0.0200 - val_loss: 612.7764
Epoch 8/50
776/776 [==============================] - 31s 40ms/step - loss: 0.0189 - val_loss: 0.3387
Epoch 9/50
776/776 [==============================] - 30s 39ms/step - loss: 0.0189 - val_l

In [13]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_2 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    x_data_temp = x_data[[f'{predict_column}_2', f'{predict_column}_3']]

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train_current, x_test_current = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        LSTM(64, input_shape=(x_train_current.shape[1], 1), activation='relu', return_sequences=True),
        BatchNormalization(),
        LSTM(32, activation='relu'),
        BatchNormalization(),
        Dense(1),
        Activation('linear'),
    ])

    print("형태 변경 전 - x_train_current 형태:", x_train_current.shape)
    x_train_lstm_current = np.reshape(x_train_current.values, (x_train_current.shape[0], x_train_current.shape[1], 1))
    x_test_lstm_current = np.reshape(x_test_current.values, (x_test_current.shape[0], x_test_current.shape[1], 1))
    print("형태 변경 후 - x_train_lstm_current 형태:", x_train_lstm_current.shape)

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(x_train_lstm_current, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

    y_pred = model.predict(x_test_lstm_current).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    x_data_lstm_current = np.reshape(x_data_temp.values, (x_data_temp.shape[0], x_data_temp.shape[1], 1))
    predicted_values_2[f'{predict_column}_1_pre'] = model.predict(x_data_lstm_current).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

lstm_pred_2 = pd.concat([x_data, predicted_values_2], axis=1)

형태 변경 전 - x_train_current 형태: (31026, 88)
형태 변경 후 - x_train_lstm_current 형태: (31026, 88, 1)
Epoch 1/50
776/776 [==============================] - 70s 85ms/step - loss: 0.2432 - val_loss: 0.1279
Epoch 2/50
776/776 [==============================] - 62s 81ms/step - loss: 0.0214 - val_loss: 0.0308
Epoch 3/50
776/776 [==============================] - 62s 81ms/step - loss: 0.0214 - val_loss: 0.0842
Epoch 4/50
776/776 [==============================] - 63s 82ms/step - loss: 0.0214 - val_loss: 0.0209
Epoch 5/50
776/776 [==============================] - 65s 84ms/step - loss: 0.0214 - val_loss: 0.1024
Epoch 6/50
776/776 [==============================] - 62s 80ms/step - loss: 0.0214 - val_loss: 0.2923
Epoch 7/50
776/776 [==============================] - 62s 80ms/step - loss: 0.0684 - val_loss: 0.0484
Epoch 8/50
776/776 [==============================] - 64s 83ms/step - loss: 0.0215 - val_loss: 0.1310
Epoch 9/50
776/776 [==============================] - 62s 80ms/step - loss: 0.0215 - val_los

In [14]:
target_columns = ['grade', 'result', 'sales', 'infra', 'franch', 'pop', 'fin']

predicted_values_3 = pd.DataFrame(index=x_data.index)
mse_dict = {}

for predict_column in target_columns:

    y_column = f'{predict_column}_y'
    y_temp = y_data[[y_column]].values.ravel()

    x_data_temp = x_data[[f'{predict_column}_2', f'{predict_column}_3']]

    total_samples = len(x_data)
    train_samples = int(0.8 * total_samples)

    x_train_current, x_test_current = x_data[:train_samples], x_data[train_samples:]
    y_train, y_test = y_temp[:train_samples], y_temp[train_samples:]

    model = Sequential([
        LSTM(64, input_shape=(x_train_current.shape[1], 1), activation='relu', return_sequences=True),
        BatchNormalization(),
        LSTM(32, activation='relu', return_sequences=True),
        BatchNormalization(),
        LSTM(32, activation='relu'),
        BatchNormalization(),
        Dense(1),
        Activation('linear'),
    ])

    print("형태 변경 전 - x_train_current 형태:", x_train_current.shape)
    x_train_lstm_current = np.reshape(x_train_current.values, (x_train_current.shape[0], x_train_current.shape[1], 1))
    x_test_lstm_current = np.reshape(x_test_current.values, (x_test_current.shape[0], x_test_current.shape[1], 1))
    print("형태 변경 후 - x_train_lstm_current 형태:", x_train_lstm_current.shape)

    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    model.fit(x_train_lstm_current, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

    y_pred = model.predict(x_test_lstm_current).reshape(-1)

    mse = mean_squared_error(y_test, y_pred)
    mse_dict[predict_column] = mse
    print(f'{predict_column} 예측 MSE: {mse}')

    x_data_lstm_current = np.reshape(x_data_temp.values, (x_data_temp.shape[0], x_data_temp.shape[1], 1))
    predicted_values_3[f'{predict_column}_1_pre'] = model.predict(x_data_lstm_current).reshape(-1)

for column, mse_value in mse_dict.items():
    print(f'{column} 예측 MSE: {mse_value}')

lstm_pred_3 = pd.concat([x_data, predicted_values_3], axis=1)

형태 변경 전 - x_train_current 형태: (31026, 88)
형태 변경 후 - x_train_lstm_current 형태: (31026, 88, 1)
Epoch 1/50
776/776 [==============================] - 98s 119ms/step - loss: 0.0989 - val_loss: 6.0562
Epoch 2/50
776/776 [==============================] - 93s 120ms/step - loss: 0.0208 - val_loss: 2.8175
Epoch 3/50
776/776 [==============================] - 96s 124ms/step - loss: 0.0197 - val_loss: 1.7776
Epoch 4/50
776/776 [==============================] - 91s 118ms/step - loss: 0.0194 - val_loss: 0.3212
Epoch 5/50
776/776 [==============================] - 93s 120ms/step - loss: 0.0195 - val_loss: 0.0582
Epoch 6/50
776/776 [==============================] - 91s 118ms/step - loss: 0.0194 - val_loss: 3.7958
Epoch 7/50
776/776 [==============================] - 91s 118ms/step - loss: 0.0194 - val_loss: 8.3442
Epoch 8/50
776/776 [==============================] - 99s 128ms/step - loss: 0.0194 - val_loss: 0.0472
Epoch 9/50
776/776 [==============================] - 91s 118ms/step - loss: 0.0193 

# 최종 데이터 저장

In [ ]:
xgb_pred.to_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/2. 상권 활성화 지수 예측 모델/3) 예측 데이터/x_forecast_xg.csv', index=False)